In [4]:
import pandas as pd

# Load the uploaded CSV file
file_path = "mental_health_lite.csv"
df = pd.read_csv(file_path)

# Display the first few rows and basic info
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     1001 non-null   int64  
 1   gender                  1001 non-null   object 
 2   employment_status       1001 non-null   object 
 3   work_environment        1001 non-null   object 
 4   mental_health_history   1001 non-null   object 
 5   seeks_treatment         1001 non-null   object 
 6   stress_level            1001 non-null   int64  
 7   sleep_hours             1001 non-null   float64
 8   physical_activity_days  1001 non-null   int64  
 9   depression_score        1001 non-null   int64  
 10  anxiety_score           1001 non-null   int64  
 11  social_support_score    1001 non-null   int64  
 12  productivity_score      1001 non-null   float64
 13  mental_health_risk      1001 non-null   object 
dtypes: float64(2), int64(6), object(6)
memor

(   age      gender employment_status work_environment mental_health_history  \
 0   56        Male          Employed          On-site                   Yes   
 1   46      Female           Student          On-site                    No   
 2   32      Female          Employed          On-site                   Yes   
 3   60  Non-binary     Self-employed          On-site                    No   
 4   25      Female     Self-employed          On-site                   Yes   
 
   seeks_treatment  stress_level  sleep_hours  physical_activity_days  \
 0             Yes             6          6.2                       3   
 1             Yes            10          9.0                       4   
 2              No             7          7.7                       2   
 3              No             4          4.5                       4   
 4             Yes             3          5.4                       0   
 
    depression_score  anxiety_score  social_support_score  productivity_score 

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# Encode categorical columns
df_encoded = df.copy()
label_encoders = {}

# Train XGBoost dengan default parameters
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)

for column in df_encoded.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_encoded[column])
    label_encoders[column] = le

# Pisahkan fitur dan target
X = df_encoded.drop("mental_health_risk", axis=1)
y = df_encoded["mental_health_risk"]

# Split data 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model tanpa tuning hyperparameter
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# Prediksi dan evaluasi
y_pred_xgb = xgb_model.predict(X_test)
report_xgb = classification_report(y_test, y_pred_xgb, target_names=label_encoders['mental_health_risk'].classes_, digits=4)
print(report_xgb)

# Evaluasi
report = classification_report(y_test, y_pred, target_names=label_encoders['mental_health_risk'].classes_, digits=4)
print(report)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:12:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

        High     1.0000    0.9630    0.9811        54
         Low     0.9655    0.8750    0.9180        32
      Medium     0.9500    0.9913    0.9702       115

    accuracy                         0.9652       201
   macro avg     0.9718    0.9431    0.9565       201
weighted avg     0.9659    0.9652    0.9648       201

              precision    recall  f1-score   support

        High     0.9792    0.8704    0.9216        54
         Low     0.9583    0.7188    0.8214        32
      Medium     0.8760    0.9826    0.9262       115

    accuracy                         0.9104       201
   macro avg     0.9378    0.8572    0.8897       201
weighted avg     0.9168    0.9104    0.9083       201

